In [ ]:
# install simpletransformers
%%capture
!pip install simpletransformers

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16843272024337924919
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11320098816
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5340594685754830012
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
 xla_global_id: 416903419]

In [ ]:
# import some necessary sub libraries
import pandas as pd
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import f1_score, accuracy_score
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
import os
import tarfile
import logging

In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv("all_tweets_predicted.csv", sep=",")
len(data)
data.tail()

,Unnamed: 0,UserScreenName,UserName,Timestamp,Text,Embedded_text,Emojis,Comments,Likes,Retweets,Image link,Tweet URL,id,Partei,sentiment_prediction
58859,10566,Tino Chrupalla,@Tino_Chrupalla,2021-12-04T17:26:46.000Z,Tino Chrupalla\n@Tino_Chrupalla\n·\n4. Dez. 2021,Friedlicher Protest gegen einen #Impfzwang ist...,NaN,265.0,122.0,578.0,[],https://twitter.com/Tino_Chrupalla/status/1467...,61802,AfD,2
58860,10567,Tino Chrupalla,@Tino_Chrupalla,2021-12-13T16:30:19.000Z,Tino Chrupalla\n@Tino_Chrupalla\n·\n13. Dez. 2021,@OlafScholz\n muss sich endlich klar zu Nord S...,NaN,4.0,2.0,11.0,[],https://twitter.com/Tino_Chrupalla/status/1470...,61803,AfD,2
58861,10568,Tino Chrupalla,@Tino_Chrupalla,2021-12-17T14:02:24.000Z,Tino Chrupalla\n@Tino_Chrupalla\n·\n17. Dez. 2021,Mit \n@_FriedrichMerz\n gibt es keine konserva...,NaN,188.0,227.0,1.008,[],https://twitter.com/Tino_Chrupalla/status/1471...,61804,AfD,1
58862,10569,Tino Chrupalla,@Tino_Chrupalla,2021-12-19T09:27:23.000Z,Tino Chrupalla\n@Tino_Chrupalla\n·\n19. Dez. 2021,Wir wünschen Ihnen und Ihrer Familie einen bes...,NaN,449.0,346.0,2.648,['https://pbs.twimg.com/media/FG9dwrcXIAUF5BP?...,https://twitter.com/Tino_Chrupalla/status/1472...,61805,AfD,0
58863,10570,Tino Chrupalla,@Tino_Chrupalla,2021-12-23T13:40:49.000Z,Tino Chrupalla\n@Tino_Chrupalla\n·\n23. Dez. 2021,@_FriedrichMerz\n will parlamentarische Zusamm...,NaN,9.0,4.0,11.0,['https://pbs.twimg.com/card_img/1481535392549...,https://twitter.com/Tino_Chrupalla/status/1474...,61806,AfD,1


In [ ]:
df = data["Embedded_text"].dropna()
#len(data["Embedded_text"].dropna())
data = data[data['Embedded_text'].notna()]
len(data)

58864

In [ ]:
def pack_model(model_path='',file_name=''):
  files = [files for root, dirs, files in os.walk(model_path)][0]
  with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
    for file in files:
      f.add(f'{model_path}/{file}')

In [ ]:
def unpack_model(model_name=''):
  tar = tarfile.open(f"{model_name}.tar.gz", "r:gz")
  tar.extractall()
  tar.close()

In [ ]:
def freq(lst):
    d = {}
    for i in lst:
        if d.get(i):
            d[i] += 1
        else:
            d[i] = 1
    return d

In [ ]:
def getPercentages(fd):
  sum = 0
  for item in fd:
    sum = sum + fd[item]
  ret = {}
  for item in fd:
    fp_dict = {}
    fp_dict["frequ"] = fd[item]
    fp_dict["percentage"] = fd[item]/sum * 100
    ret[item] = fp_dict
  return ret

In [ ]:
def getResults(input):
  return getPercentages(freq(input))

In [ ]:
path_model = path_model_tt = "/content/drive/MyDrive/DH_KFOLD/mitGermEvalaberAndererAnsatzSehrGUT_4"

In [ ]:
# define hyperparameter
train_args ={"reprocess_input_data": True,
             "fp16":False,
             "num_train_epochs": 4,
             "overwrite_output_dir":True,
             "train_batch_size": 32,
             "eval_batch_size": 32}

In [ ]:
#mec
unpack_model(path_model)

model = ClassificationModel(
    "bert", "content/outputs/",
    num_labels= 3,
    args=train_args
)

In [ ]:
model.predict(["Ich liebe dich."])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

([2], array([[-2.39702439, -3.17064881,  4.80356359]]))

In [ ]:
model.predict(["Ich hasse dich!"])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

([1], array([[-3.50486779,  4.29590416, -0.90006942]]))

In [ ]:
model.predict(["Corona Maßnahmen"])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

([2], array([[-3.20347714, -2.73739576,  4.98413229]]))

In [ ]:
texts = []
for index, row in data.iterrows():
  texts.append(row["Embedded_text"])

In [ ]:
len(texts)

58864

In [ ]:
output = model.predict(texts)

  0%|          | 0/58864 [00:00<?, ?it/s]

  0%|          | 0/1840 [00:00<?, ?it/s]

In [ ]:
len(output[0])

58864

In [ ]:
# 0 = positiv, 1 = negativ, 2 = neutral
data["sentiment_prediction"] = output[0]

In [ ]:
data.to_csv("all_tweets_predicted_bert93.csv")

In [ ]:
print(data["Embedded_text"][4], data["sentiment_prediction"][4])

Gendern geht („*innen“)
Impfen geht nicht.
Diese Regierung blamiert Deutschland in der Welt.
Abwählen!
BMF
@BMF_Bund
 · 7. März
Ab Ende des Monats werden wir pro Woche Millionen Bürger*innen impfen können. Habe dafür gesorgt, dass das vorbereitet wird. So Bundesfinanzminister ⁦@OlafScholz⁩ ⁦@berlindirekt⁩ zur Frage, ob es mit #Impfstrategie klappt, wenn jetzt mehr #Impfstoff verfügbar ist.
1
3
16 1


In [ ]:
check = data[["Embedded_text", "sentiment_prediction"]]

In [ ]:
pd.set_option('display.max_colwidth',1000)

In [ ]:
pos = check[check["sentiment_prediction"] == 0]
pos

,Embedded_text,sentiment_prediction
10,Danke Polizei. \n\n#Rechtsstaat #dd1004 #AfD\n4\n2\n61,0
22,"Sympathische Polizisten.\nLinke, Grüne, SPD und andere Extremisten sind not amused.\n0:51\n423.396 Mal angezeigt\nVon \nAlexander Fröhlich\n11\n10\n35",0
23,"Sympathische Polizisten.\nLinke, Grüne, SPD und andere Extremisten sind not amused.\n0:49\n423.396 Mal angezeigt\nVon \nAlexander Fröhlich\n11\n10\n35",0
29,Steuerweltmeister Deutschland.\n\nNormal ist das nicht.\n\nDamit auch Ihr Portmonee wieder aufatmet.\n\nIm Herbst sind Wahlen.\nWELT\n@welt\n · 29. Apr.\nSteuern: Niemand auf der Welt zahlt mehr als der deutsche Single\nhttp://to.welt.de/1LyYCRy\n10\n31\n119,0
44,"CDU wählen macht arm. \nGrüne wählen auch. \nFDP auch. \nSPD und Linke sowieso. \n\nDas Gute ist: Es gibt immer eine #Alternative.\nHasso Mansfeld\n@rasenspiesser\n · 12. Mai\nDie @CDU stimmt allen Ernstes einem Gesetz zu, in dem Vermieter ihren Gewerbemietern die Hälfte der CO2-Heizungskosten zahlen sollen. Auch wenn die Vermieter gerade saniert haben. #CDU ihr seid komplett durch!\n2\n7\n28",0
...,...,...
58840,Es reicht! Frau #Merkel übernehmen Sie Verantwortung! Machen Sie den Weg frei für die Parlamentarische #Demokratie und treten Sie zurück! Zum Wohle #Deutschlands!\n214\n329\n1.596,0
58842,"Experten setzen auf ein Comeback der Kernkraft. Denn #Kernenergietechnik steht für sichere Technik, ist bezahlbar und sichert den Bürgern günstig die Stromerzeugung auf Jahrzehnte. Wir haben die besseren Konzepte für Deutschland! #AufbruchfürDeutschland\nbild.de\nExperten fordern es für den Klimaschutz - Was für ein Kernkraft-Comeback spricht\nExperten warnen: Ohne die Kernenergie gibt es auch keinen Klimaschutz. \n48\n98\n422",0
58848,"Wir danken der Republik #Polen und seinen Sicherheitskräften, die mit ihrem Einsatz, EU-Grenzen und damit die Sicherheit Deutschlands verteidigen. Die deutsche Politik ist in der Pflicht, den Magneten der Migration, nämlich die hohen Geldleistungen für #Asylbewerber, abzustellen.\n86\n311\n1.545",0
58860,@OlafScholz\n muss sich endlich klar zu Nord Stream 2 bekennen: für deutsche Interessen und damit für eine gesicherte und günstige Energieversorgung.\n4\n2\n11,0


In [ ]:
len(output[0])

58864

In [ ]:
getResults(output[0])

{0: {'frequ': 14942, 'percentage': 25.38393585213373},
 1: {'frequ': 20007, 'percentage': 33.98851590106007},
 2: {'frequ': 23915, 'percentage': 40.6275482468062}}

In [ ]:
freq(output[0])

{0: 14942, 1: 20007, 2: 23915}

In [ ]:
getPercentages(freq(output[0]))

{0: {'frequ': 14942, 'percentage': 25.38393585213373},
 1: {'frequ': 20007, 'percentage': 33.98851590106007},
 2: {'frequ': 23915, 'percentage': 40.6275482468062}}